# POS Tagging for VOC

This first cell will install the required packages on your computer. You only need to run this once, once they are installed they don't need to be installed again

In [ ]:
!pip install pympi-ling
!pip install pandas
!pip install stanza

These are the imports necessary to run the rest of the cells. Cells from here down should be run in sequence.

In [1]:
from pympi import Eaf, Praat
import pandas as pd
from pathlib import Path

In the following cell, replace the text in quotations the path in your computer where your .eaf or .TextGrid files are located.

In [7]:
path_to_transcripts = Path("<path to folder where your recordings are located>")
path_to_transcripts = Path("/users/anton/Desktop")

Here, we define a couple of functions.
QUESTION FOR ADOLFO: Do we want to add a filename key to the dictionary?

In [11]:
def format_annotations(tier_name, annotation_list, output_list):
    """This function formats a tuple of the form (begin, end, value)
    into a list of dictionaries of the form {speaker, start, end, text}"""
    
    for start_ms, end_ms, text in annotation_list:

        output_list.append({
            'speaker': tier_name,
            'start_ms': start_ms,
            'end_ms': end_ms,
            'text': text
        })
    
    return output_list


def get_annotations(root_path, filename, annotations=list(), tier_name=None):
    """Extracts all annotations for a given .eaf
    file for a given tier (if tier_name specified) or for all tiers"""
    # Load elan file
    elan_object = Eaf(file_path=root_path / filename)
    
    # If a tier name is provided, only annotations for that
    # tier will be used.
    if tier_name:
        tiernames = [tier_name]
    else:
        # Get all tier names
        tiernames = elan_object.get_tier_names()

    # Iterate over all tiers and extract annotations
    for tier in tiernames:
        
        annotations = format_annotations(
            tier,
            elan_object.get_annotation_data_for_tier(tier),
            annotations
            )
    return annotations
    

def iterate_over_folder(
    root_path, annotations=list(),
    file_extension='.eaf', # This parameter can't really be changes with the code as is.
    output_format='dict_list',
    tier_name=None
    ):
    """Iterates over files of a specified extensions and 
    extracts annotations into either a list of dictionaries
    or a pandas dataframe."""
    for file in root_path.glob(f'*{file_extension}'):
        
        annotations = get_annotations(
            root_path, file,
            annotations=annotations,
            tier_name=tier_name
            )
    
    
    if output_format == 'pandas':
        return pd.DataFrame(annotations)
    elif output_format == 'dict_list':
        return annotations


dict_keys(['AMA_Stepney_SarahA', 'Interviewer'])


,speaker,start_ms,end_ms,text
0,AMA_Stepney_SarahA,0,2294,"so being .. introduced like an actual,"
1,AMA_Stepney_SarahA,270165,270655,"yeah,"
2,AMA_Stepney_SarahA,272560,273470,"yeah,"
3,AMA_Stepney_SarahA,282355,285675,uhm yeah they all- they all came from uh-
4,AMA_Stepney_SarahA,285685,286305,"uhm,"
5,AMA_Stepney_SarahA,286535,288065,"families that had a lot of money,"
6,AMA_Stepney_SarahA,288370,290980,uh: you couldn't really tell though cuz they a...
7,AMA_Stepney_SarahA,291205,292555,@but @yeah
8,AMA_Stepney_SarahA,292610,294870,yeah so you couldn't really tell but they all ...
9,AMA_Stepney_SarahA,294960,298910,their families own land have businesses all th...
